<img src="Bilder/ost_logo.png" width="240" align="right"/>
<div style="text-align: left"> <b> Applied Neural Networks | FS 2025 </b><br>
<a href="mailto:christoph.wuersch@ost.ch"> © Christoph Würsch, François Chollet </a> </div>
<a href="https://www.ost.ch/de/forschung-und-dienstleistungen/technik-neu/systemtechnik/ice-institut-fuer-computational-engineering"> Eastern Switzerland University of Applied Sciences OST | ICE </a>

[![Run in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ChristophWuersch/AppliedNeuralNetworks/blob/main/ANN09/9.2-Word_Embeddings_pl.ipynb)

This notebook contains the first code sample found in Chapter 6, Section 1 of [Deep Learning with Python](https://www.manning.com/books/deep-learning-with-python?a_aid=keras&a_bid=76564dff).

In [ ]:
# für Ausführung auf Google Colab auskommentieren und installieren
!pip install -q -r https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/requirements.txt

# 9.2 Word Embeddings, GloVe and Text classification
In diesem Notebook werden wir die Konzepte und die Verwendung von Worteinbettungen in NLP am Beispiel von Glove erklären. Dann werden wir versuchen, die vortrainierten Glove-Worteinbettungen anzuwenden, um ein Textklassifizierungsproblem mit dieser Technik zu lösen.


- Die Verwendung von **dichtbesetzten Wortvektoren** ist eine weitere verbreitete und leistungsfähige Methode, einem Wort einen Vektor zuzuordnen. 
- Man spricht hier auch von einer Worteinbettung. 
- Die von der One-hot-Codierung erzeugten Vektoren sind binär, **dünnbesetzt (enthalten vorwiegend Nullen) und sehr hochdimensional** (die Dimensionalität entspricht der Anzahl der Wörter des Vokabulars). 
- Worteinbettungen (word embeddings) hingegen sind niedrigdimensionale **Fliesskommazahlvektoren, also** vergleichsweise
dichtbesetzt:

<img src="Bilder/word_embeddings.png" width="440" height="440" align="center"/>

*Die durch One-hot-Codierung oder Hashing erzeugten Wortrepräsentationen sind dünnbesetzt, hochdimensional und fest einprogrammiert. Worteinbettungen dagegen sind dichtbesetzt, relativ niedrigdimensional und anhand der Daten erlernt.*


### Word Embeddings trainieren

- Im Gegensatz zu den durch One-hot-Codierung erzeugten Wortvektoren werden **Word Embeddings anhand der Daten erlernt**. 
- Worteinbettungen sind nicht selten 256-, 512- oder 1.024-dimensional, wenn das Vokabular entsprechend gross ist.
- Andererseits ergeben sich bei der One-hot-Codierung oft Vektoren, die 20.000- dimensional (in diesem Fall kann ein Vokabular von 20.000 Wörtern erfasst werden) oder noch grösser sind. Worteinbettungen bringen also mehr Informationen in sehr viel weniger Dimensionen unter.

Es gibt **zwei Möglichkeiten**, Worteinbettungen zu erzeugen:

- Lassen Sie das Modell die Worteinbettungen erlernen, während die eigentliche Hauptaufgabe erledigt wird, wie etwa die Klassifizierung von Dokumenten oder die Vorhersage von Stimmungslagen. Anfangs ergeben sich zufällige Wortvektoren, später werden die Wortvektoren auf die gleiche Weise erlernt wie die Gewichtungen eines NNs.
- Lesen Sie in Ihr Modell Worteinbettungen ein, die durch andere Machine-Learning- Aufgaben berechnet worden sind. Man bezeichnet diese als vortrainierte Worteinbettungen.

Wir werden beide Methoden betrachten.

## Worteinbettungen mit dem `Embedding` Layer erlernen


- Die einfachste Methode, einem Wort einen dichtbesetzten Vektor zuzuordnen, ist die Auswahl eines *zufälligen Vektors*. 
- Dieser Ansatz bringt jedoch das Problem mit sich, dass der resultierende Einbettungsraum *keine Struktur* besitzt.
- Beispielsweise hätten die Wörter toll und super womöglich völlig verschiedene Einbettungen, obwohl sie in den meisten Sätzen austauschbar sind. Für ein NN ist es schwierig, mit einem solchen verrauschten und strukturlosen Einbettungsraum zurechtzukommen.




Betrachten wir das Ganze etwas abstrakter: 
- **Die geometrischen Beziehungen zwischen Wortvektoren sollten die semantischen Beziehungen zwischen den Wörtern widerspiegeln.** 
- Wortvektoren haben die Aufgabe, die menschliche Sprache auf einen geometrischen Raum abzubilden.
- Bei einem vernünftigen Einbettungsraum würden Sie beispielsweise erwarten, dass *Synonyme in ähnlichen Wortvektoren eingebettet* sind. 
- Und im Allgemeinen würden Sie erwarten, dass der *geometrische Abstand (wie die L2-Norm) zwischen zwei beliebigen Wortvektoren in einer bestimmten Beziehung zum semantischen Abstand der dazugehörigen Wörter steht*: Wörter mit ganz verschiedenen Bedeutungen sind an weit voneinander entfernten Positionen eingebettet, während sich miteinander verwandte Wörter näher sind. 
- Neben dem Abstand könnte man auch bestimmten *Richtungen im Einbettungsraum* eine Bedeutung verleihen wollen. Betrachten wir zur Verdeutlichung ein konkretes Beispiel.

Die folgende Abbildung zeigt vier in eine zweidimensionale Ebene eingebettete Wörter: Katze, Hund, Wolf und Tiger. 

Mit den hier gewählten Vektorrepräsentationen können bestimmte semantische Beziehungen zwischen diesen Wörtern als geometrische
Abbildungen codiert werden. 
- So führt beispielsweise der gleiche Vektor sowohl von Katze zu Tiger als auch von Hund zu Wolf. Man könnte diesen Vektor als »vom Haustier zum Wildtier« interpretieren. 
- Auf ähnliche Weise führt ein weiterer Vektor von Hund zu Katze bzw. von Wolf zu Tiger, den man wiederum als »von hundeartig zu katzenartig« interpretieren kann.

Für in der Praxis auftretende Worteinbettungsräume sind »Geschlecht«- und »Plural«-Vektoren typische Beispiele für sinnvolle geometrische Abbildungen. 
- Wenn man beispielsweise zum Vektor »König« den Vektor »weiblich« hinzuaddiert, ergibt sich »Königin«. 
- Durch das Hinzuaddieren eines »Plural«-Vektors erhält man »Könige«. 
- Worteinbettungsräume enthalten typischerweise Tausende dieser interpretierbaren und potenziell nützlichen Vektoren.

In [ ]:
from IPython.display import display
from PIL import Image
import requests

display(
    Image.open(
        requests.get(
            "https://raw.githubusercontent.com/ChristophWuersch/AppliedNeuralNetworks/main/ANN09/Bilder/Beispiel_Word_Embedding.PNG",
            stream=True,
        ).raw
    ).convert("RGB")
)



Daher ist es vernünftig, bei jeder neuen Aufgabe auch einen neuen Worteinbettungsraum
zu erlernen. 

# Grosser Filmkritik-Datensatz
Dies ist ein Datensatz für die binäre Stimmungs-Klassifikation, der wesentlich mehr Daten enthält als frühere Benchmark-Datensätze. Wir stellen einen Satz von 25.000 hochpolaren Filmkritiken zum Training und 25.000 zum Testen zur Verfügung. Es gibt auch zusätzliche unbeschriftete Daten zur Verwendung. Es werden Rohtext und bereits verarbeitete Bag of Words-Formate bereitgestellt. Weitere Einzelheiten finden Sie in der README-Datei, die in der Veröffentlichung enthalten ist.

Link zum Datensatz: http://ai.stanford.edu/~amaas/data/sentiment/


## Laden die Libraries


In [ ]:
! pip install --quiet kagglehub

import kagglehub
import zipfile
import os
import shutil


def download_and_extract_glove_to_data(data_dir="data"):
    # Erstelle Zielordner falls nötig
    os.makedirs(data_dir, exist_ok=True)

    # Lade GloVe über kagglehub (landet im Cache-Ordner)
    cached_path = kagglehub.dataset_download("sawarn69/glove6b100dtxt")
    print("Heruntergeladen in Cache:", cached_path)

    # Kopiere den Inhalt des Cache-Ordners nach data/
    for item in os.listdir(cached_path):
        source_path = os.path.join(cached_path, item)
        target_path = os.path.join(data_dir, item)

        # Wenn es eine ZIP-Datei ist, entpacken
        if item.endswith(".zip"):
            with zipfile.ZipFile(source_path, "r") as zip_ref:
                zip_ref.extractall(data_dir)
            print(f"Entpacke ZIP-Datei: {item}")
        else:
            # Ansonsten einfach die Datei/Ordner verschieben
            print(f"Kopiere {source_path} nach {target_path}...")
            shutil.copy(source_path, target_path)

    print("Fertig: GloVe-Dateien sind jetzt in", data_dir)


# Aufruf
download_and_extract_glove_to_data()


In [ ]:
! pip install gensim

In [ ]:
# === Zusätzliche Imports ===
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

%matplotlib inline

# Import module to split the datasets
from sklearn.model_selection import train_test_split

# Import modules to evaluate the metrics
from sklearn import metrics
from sklearn.metrics import (
    confusion_matrix,
    accuracy_score,
    roc_auc_score,
    roc_curve,
    auc,
)

# Wir brauchen diese Funktion einmalig, um das GloVe-Format in Word2Vec-Format zu konvertieren
from gensim.scripts.glove2word2vec import glove2word2vec

# === Globale Parameter ===
glove_filename = "glove.6B.100d.txt"
train_filename = "train.csv"

# Verzeichnispfade
DATA_PATH = "data"
glove_path = os.path.join(DATA_PATH, glove_filename)

train_path = DATA_PATH
test_path = DATA_PATH

# Relevante Spalten
TEXT_COLUMN = "text"
TARGET_COLUMN = "target"


# Word Embedding 
Was ist ein Word Embedding? Word Embeddings sind eine Art der Wortdarstellung, die es ermöglicht, dass Wörter mit ähnlicher Bedeutung eine ähnliche Darstellung erhalten. Es handelt sich um eine verteilte Darstellung für Text, die vielleicht einer der wichtigsten Durchbrüche für die beeindruckende Leistung von Deep-Learning-Methoden bei anspruchsvollen Problemen der natürlichen Sprachverarbeitung ist.

Word Embeddings sind in der Tat eine Klasse von Techniken, bei denen einzelne Wörter als reellwertige Vektoren in einem vordefinierten Vektorraum dargestellt werden. Jedes Wort wird einem Vektor zugeordnet, und die Vektorwerte werden auf eine Art und Weise erlernt, die einem neuronalen Netz ähnelt, weshalb diese Technik oft in den Bereich des Deep Learning eingeordnet wird. Die verteilte Darstellung wird auf der Grundlage der Verwendung von Wörtern gelernt. Dies ermöglicht es, dass Wörter, die auf ähnliche Weise verwendet werden, ähnliche Repräsentationen ergeben, die ihre Bedeutung auf natürliche Weise erfassen.

Die Einbettung von neuronalen Netzen hat 3 Hauptzwecke:

- Die Suche nach den nächsten Nachbarn im Einbettungsraum. Diese können verwendet werden, um Empfehlungen auf der Grundlage von Benutzerinteressen oder Clusterkategorien auszusprechen.
- Als Input für ein maschinelles Lernmodell für eine überwachte Aufgabe.
- Zur Visualisierung von Konzepten und Beziehungen zwischen Kategorien.

Wie können wir die Word Embedding erhalten?

## Embedding Layer
Eine Einbettungsschicht ist eine Worteinbettung, die in einem neuronalen Netzmodell für eine bestimmte Aufgabe der natürlichen Sprachverarbeitung gelernt wird. Die Dokumente oder der Korpus der Aufgabe werden gesäubert und aufbereitet, und die Größe des Vektorraums wird als Teil des Modells festgelegt, beispielsweise 50, 100 oder 300 Dimensionen. Die Vektoren werden mit kleinen Zufallszahlen initialisiert. Die Einbettungsschicht wird am vorderen Ende eines neuronalen Netzes verwendet und wird mit Hilfe des Backpropagation-Algorithmus auf überwachte Weise angepasst.

Es handelt sich um eine flexible Schicht, die auf verschiedene Weise verwendet werden kann, z. B:

- Sie kann allein verwendet werden, um eine Worteinbettung zu lernen, die gespeichert und später in einem anderen Modell verwendet werden kann.
- Sie kann als Teil eines Deep-Learning-Modells verwendet werden, wobei die Einbettung zusammen mit dem Modell selbst gelernt wird.
- Es kann verwendet werden, um ein vortrainiertes Worteinbettungsmodell zu laden, eine Art von Transfer-Lernen.
Dieser Ansatz erfordert eine große Menge an Trainingsdaten und kann sehr langsam sein, lernt aber eine Einbettung, die sowohl auf die spezifischen Textdaten als auch auf die gewünschte NLP-Aufgabe zugeschnitten ist.

## Word2Vec

Word2Vec ist eine statistische Methode zum effizienten Lernen einer eigenständigen Worteinbettung aus einem Textkorpus. Sie wurde 2013 von Tomas Mikolov et al. bei Google entwickelt, um das auf neuronalen Netzen basierende Training der Einbettung effizienter zu gestalten, und ist seitdem zum De-facto-Standard für die Entwicklung von vortrainierten Worteinbettungen geworden.

Es lernt die Worteinbettung auf eine von zwei Arten:

- entweder anhand des Kontexts, um ein Zielwort vorherzusagen, eine Methode, die als Continuous Bag of Words (CBOW) bekannt ist
- oder die Verwendung eines Wortes zur Vorhersage eines Zielkontextes, was als Skip-Gram bezeichnet wird, z. B. möchten wir c Kontextwörter vorhersagen, die ein Zielwort in der Eingabe haben. Die letztere Methode liefert in der Regel genauere Ergebnisse bei großen Datensätzen.

<img src="Bilder/CBOW-model.png" width="700"  align="center"/>

Ein gut trainierter Satz von Wortvektoren wird ähnliche Wörter nahe beieinander in diesem Raum platzieren. Die Wörter „Eiche“, „Ulme“ und „Birke“ könnten sich in einer Ecke anhäufen, während „Krieg“, „Konflikt“ und „Streit“ sich in einer anderen Ecke aneinander drängen.

## Glove
Der GloVe-Algorithmus (Global Vectors for Word Representation) ist eine Erweiterung der word2vec-Methode zum effizienten Lernen von Wortvektoren, die von Pennington et al. in Stanford entwickelt wurde. GloVe ist ein unüberwachter Lernalgorithmus zur Gewinnung von Vektordarstellungen für Wörter. Das Training erfolgt auf der Grundlage aggregierter globaler Wort-Wort-Ko-Okzidenz-Statistiken aus einem Korpus, und die resultierenden Repräsentationen zeigen interessante lineare Unterstrukturen des Wortvektorraums.

GloVe ist ein Ansatz, der sowohl die globalen Statistiken von Matrixfaktorisierungsverfahren wie LSA (Latent Semantic Analysis) mit dem lokalen kontextbasierten Lernen in word2vec verbindet. Anstatt ein Fenster zu verwenden, um den lokalen Kontext zu definieren, konstruiert GloVe eine explizite Wort-Kontext- oder Wort-Ko-Occurrence-Matrix unter Verwendung von Statistiken über den gesamten Textkorpus.

Eine ausführliche Erklärung finden Sie unter dem nächsten Link: Jeffrey Pennington, Richard Socher, und Christopher D. Manning. 2014. GloVe: Global Vectors for Word Representation. https://nlp.stanford.edu/pubs/glove.pdf

## Laden einer vortrainierten Worteinbettung: GloVe
Dateien mit den vortrainierten Vektoren Glove finden Sie auf vielen Seiten wie Kaggle oder unter dem oben genannten Link der Stanford University. Wir werden die Datei ``glove.6B.100d.txt`` verwenden, die die Glove-Vektoren enthält, die auf dem Wikipedia- und GigaWord-Datensatz trainiert wurden.

GloVe und Word2Vec sind zwei unterschiedliche Formate für vortrainierte Wortvektoren.
- Viele Tools (wie `gensim`) erwarten Vektoren im Word2Vec-Format (also mit Headerzeile und Leerzeichen als Trennzeichen).
- GloVe-Dateien (z. B. `glove.6B.300d.txt`) haben keine Headerzeile und sind durch Leerzeichen getrennt, aber nicht direkt kompatibel mit dem Word2Vec-Ladeformat.


In [ ]:
glove_filename = "glove.6B.100d.txt"  # Beispielname
glove_path = f"data/{glove_filename}"
word2vec_output_file = f"data/{glove_filename}.word2vec"

# Nur konvertieren, wenn die Datei noch nicht existiert
if not os.path.exists(word2vec_output_file):
    print(f"Konvertiere {glove_filename} in Word2Vec-Format...")
    glove2word2vec(glove_path, word2vec_output_file)
    print("Konvertierung abgeschlossen.")
else:
    print(f"Datei {word2vec_output_file} existiert bereits. Überspringe Konvertierung.")

Unser Vokabular enthält also 400K Wörter, die durch einen Merkmalsvektor der Form 100 dargestellt werden. Jetzt können wir die Glove-Einbettungen im word2vec-Format laden und dann einige Analogien analysieren. Wenn wir auf diese Weise eine bereits trainierte word2vec-Einbettung verwenden wollen, können wir einfach den Dateinamen ändern und den gesamten unten stehenden Code wiederverwenden.

In [ ]:
from gensim.models import KeyedVectors

# load the Stanford GloVe model
word2vec_output_file = "data/" + glove_filename + ".word2vec"
model = KeyedVectors.load_word2vec_format(word2vec_output_file, binary=False)

# Show a word embedding
print("King: ", model.get_vector("king"))

result = model.most_similar(positive=["woman", "king"], negative=["man"], topn=1)

print("Most similar word to King + Woman: ", result)


## Analysieren des Vektorraums und Finden von Analogien
Da unsere Wörter numerische Vektoren sind, können wir die Abstände zwischen den Wörtern messen und vergleichen, um einige der Eigenschaften zu zeigen, die diese Einbettungen bieten.

Zum Beispiel können wir einige Analogien vergleichen. Die bekannteste ist die folgende: König - Mann + Frau = Königin. Mit anderen Worten: Die Addition der Vektoren, die mit den Wörtern König und Frau verbunden sind, und die Subtraktion von Mann ist gleich dem Vektor, der mit Königin verbunden ist. Mit anderen Worten, wenn wir den Begriff „Mann“ vom Begriff „König“ subtrahieren, erhalten wir eine Darstellung der „Königswürde“. Wenn wir dann das Wort „Frau“ zu diesem Konzept addieren, erhalten wir das Wort „Königin“. Ein anderes Beispiel ist: Frankreich - Paris + Rom = Italien. In diesem Fall erfasst die Vektordifferenz zwischen Paris und Frankreich das Konzept des Landes.

Nun werden wir einige dieser Analogien in verschiedenen Themenbereichen aufzeigen.

In [ ]:
result = model.most_similar(positive=["woman", "king"], negative=["man"], topn=1)
print("King - Man + Woman = ", result)
result = model.most_similar(positive=["rome", "france"], negative=["paris"], topn=1)
print("France - Paris + Rome = ", result)
result = model.most_similar(positive=["english", "france"], negative=["french"], topn=1)
print("France - french + english = ", result)
result = model.most_similar(
    positive=["june", "december"], negative=["november"], topn=1
)
print("December - November + June = ", result)
result = model.most_similar(positive=["sister", "man"], negative=["woman"], topn=1)
print("Man - Woman + Sister = ", result)


Wir können beobachten, wie die Wortvektoren Informationen enthalten, um Länder mit Nationalitäten, Monaten des Jahres, Familienbeziehungen usw. in Verbindung zu bringen.

Aber nicht immer erhalten wir die erwarteten Ergebnisse:

In [ ]:
# But not always we get the expected result
result = model.most_similar(positive=["aunt", "nephew"], negative=["niece"], topn=1)
print("France - Paris + Rome = ", result)


Wir können extrahieren, welche Wörter einem anderen Wort am ähnlichsten sind, sodass sie im Vektorraum "sehr nah" beieinander liegen.

In [ ]:
result = model.most_similar(positive=["spain"], topn=10)
print("10 most similar words to Spain: ", result)

result = model.most_similar(positive=["football"], topn=10)
print("\n10 most similar words to Football: ", result)

result = model.most_similar(positive=["doctor"], topn=10)
print("\n10 most similar words to Doctor: ", result)


In [ ]:
# Lets show some measure of similarities between words
result = model.similar_by_word("cat")
print(" Cat is similar to {}: {:.4f}".format(*result[0]))
result = model.similar_by_word("father")
print(" Father is similar to {}: {:.4f}".format(*result[0]))


Die Idee hinter allen Worteinbettungen ist es, mit ihnen so viele semantische/morphologische/kontextuelle/hierarchische/etc. Informationen wie möglich zu erfassen, aber in der Praxis ist eine Methode definitiv besser als die andere für eine bestimmte Aufgabe (z.B. ist LSA ziemlich effektiv, wenn man im niedrigdimensionalen Raum für die Analyse von eingehenden Dokumenten aus dem gleichen Bereich arbeitet wie die, die bereits verarbeitet und in die Term-Dokument-Matrix eingefügt wurden). Das Problem der Auswahl der besten Einbettungen für ein bestimmtes Projekt ist immer das Problem des Versuch-und-Irrtum-Ansatzes, so dass das Erkennen, warum in einem bestimmten Fall ein Modell besser funktioniert als das andere, bei der realen Arbeit ausreichend hilft.

## Visualisierung von Worteinbettungen
Ein weiterer spannender Vorgang, den wir mit Einbettungen durchführen können, ist die Visualisierung. Wenn wir sie in einem zweidimensionalen Raum aufzeichnen, können wir sehen, wie die Wörter miteinander verwandt sind. Die meisten ähnlichen Wörter sollten in Gruppen dargestellt werden, während nicht verwandte Wörter in einem großen Abstand erscheinen. Dies erfordert eine weitere Dimensionsreduktionstechnik, um die Dimensionen auf 2 oder 3 zu reduzieren. Die beliebteste Technik zur Dimensionsreduktion ist selbst eine Einbettungsmethode: t-Distributed Stochastic Neighbor Embedding (TSNE).

t-SNE steht für t-distributed stochastic neighbor embedding (t-verteilte stochastische Nachbarschaftseinbettung). Es handelt sich um ein Verfahren zur Dimensionalitätsreduktion, das sich am besten für die Visualisierung hochdimensionaler Datensätze eignet.

TSNE ist eine Technik des vielfältigen Lernens, d. h. es wird versucht, hochdimensionale Daten auf eine niedrigdimensionale Mannigfaltigkeit abzubilden und eine Einbettung zu schaffen, die versucht, die lokale Struktur innerhalb der Daten zu erhalten. Es wird fast ausschließlich für die Visualisierung verwendet, da die Ausgabe stochastisch ist und die Umwandlung neuer Daten nicht unterstützt wird.

In [ ]:
from sklearn.manifold import TSNE  # final reduction
import numpy as np  # array handling


def display_closestwords_tsnescatterplot(model, dim, words):
    arr = np.empty((0, dim), dtype="f")
    word_labels = words

    # get close words
    # close_words = [model.similar_by_word(word) for word in words]

    # add the vector for each of the closest words to the array
    close_words = []
    for word in words:
        arr = np.append(arr, np.array([model[word]]), axis=0)
        close_words += model.similar_by_word(word)

    for wrd_score in close_words:
        wrd_vector = model[wrd_score[0]]
        word_labels.append(wrd_score[0])
        arr = np.append(arr, np.array([wrd_vector]), axis=0)

    # find tsne coords for 2 dimensions
    tsne = TSNE(n_components=2, random_state=0)
    # np.set_printoptions(suppress=True)
    Y = tsne.fit_transform(arr)

    x_coords = Y[:, 0]
    y_coords = Y[:, 1]
    # display scatter plot
    plt.scatter(x_coords, y_coords)

    for label, x, y in zip(word_labels, x_coords, y_coords):
        plt.annotate(label, xy=(x, y), xytext=(0, 0), textcoords="offset points")
    plt.xlim(x_coords.min() + 0.00005, x_coords.max() + 0.00005)
    plt.ylim(y_coords.min() + 0.00005, y_coords.max() + 0.00005)
    plt.show()


def tsne_plot(model, words):
    "Creates and TSNE model and plots it"
    labels = []
    tokens = []

    # for word in model.wv.vocab:
    for word in words:
        tokens.append(model[word])
        labels.append(word)

    tsne_model = TSNE(
        perplexity=10, n_components=2, init="pca", n_iter=2500, random_state=23
    )
    new_values = tsne_model.fit_transform(np.array(tokens))

    x = []
    y = []
    for value in new_values:
        x.append(value[0])
        y.append(value[1])

    plt.figure(figsize=(14, 10))
    for i in range(len(x)):
        plt.scatter(x[i], y[i])
        plt.annotate(
            labels[i],
            xy=(x[i], y[i]),
            xytext=(5, 2),
            textcoords="offset points",
            ha="right",
            va="bottom",
        )
    plt.show()


Wir werden zum Beispiel hundert Wörter aus unseren Worteinbettungen und auch die ähnlichsten Wörter zu den Begriffen Frau und Auto einzeichnen. Alle ähnlichen Wörter sollten dicht beieinander liegen, während die restlichen Wörter über den Vektorraum verteilt erscheinen:

In [ ]:
# display_closestwords_tsnescatterplot(model, 100, ['man', 'dog'])
# words = model.index_to_key
words1 = model.similar_by_word("food", topn=30)
words2 = model.similar_by_word("woman", topn=30)
words3 = model.similar_by_word("car", topn=30)
words = [w[0] for w in words1] + [w[0] for w in words2] + [w[0] for w in words3]
# print(words)
tsne_plot(model, words)


In [ ]:
np.shape(words)

In der obigen Abbildung ist zu erkennen, dass die ähnlichsten Wörter zu „Frau“ und „Auto“ sehr nahe beieinander liegen, während die anderen Wörter gleichmäßig im Vektorraum verteilt sind.

## Laden des Datensatzes

In [ ]:
import tarfile
import urllib


# Lade und extrahiere IMDb-Daten
def download_and_extract_imdb(data_dir="data", dataset_folder="aclImdb"):
    data_path = os.path.join(data_dir, dataset_folder)
    archive_path = os.path.join(data_dir, "aclImdb_v1.tar.gz")

    if not os.path.exists(data_path):
        url = "https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
        print("Downloading IMDb dataset...")

        import ssl

        ssl._create_default_https_context = ssl._create_unverified_context

        os.makedirs(data_dir, exist_ok=True)
        urllib.request.urlretrieve(url, archive_path)

        with tarfile.open(archive_path, "r:gz") as tar:
            tar.extractall(path=data_dir, filter="data")

        print("Dataset downloaded and extracted to:", data_path)
    else:
        print("IMDb dataset already exists at:", data_path)


# Lese IMDb-Dateien
def load_imdb_dataset(path, sentiment):
    data = []
    labels = []
    folder = os.path.join(path, sentiment)
    for filename in os.listdir(folder):
        with open(os.path.join(folder, filename), encoding="utf-8") as f:
            data.append(f.read())
            labels.append(1 if sentiment == "pos" else 0)
    return data, labels


# Download IMDb-Daten
download_and_extract_imdb()

# Lade Trainings- und Testdaten
train_pos, y_train_pos = load_imdb_dataset("data/aclImdb/train", "pos")
train_neg, y_train_neg = load_imdb_dataset("data/aclImdb/train", "neg")
test_pos, y_test_pos = load_imdb_dataset("data/aclImdb/test", "pos")
test_neg, y_test_neg = load_imdb_dataset("data/aclImdb/test", "neg")

X_train_raw = train_pos + train_neg
Y_train = y_train_pos + y_train_neg
X_test_raw = test_pos + test_neg
Y_test = y_test_pos + y_test_neg


In [ ]:
import os
import urllib.request
from gensim.models import KeyedVectors

# Zielpfad für die heruntergeladene Datei
output_path = "data/GoogleNews-vectors-negative300.bin.gz"

# URL der Datei
url = "https://huggingface.co/spaces/CS482Project/Milestone_5-Search_UI_and_pitch_video_voiceover/resolve/f73493055a3f08700fae815fbbd1ca0511e29a04/Data/GoogleNews-vectors-negative300.bin.gz"

# Lade Word2Vec (z.B. GoogleNews)
print("Lade Word2Vec-Modell (kann ein paar Minuten dauern)...")

# Prüfe, ob Datei schon existiert
if not os.path.exists(output_path):
    print("Datei nicht gefunden – lade sie herunter...")
    os.makedirs("data", exist_ok=True)
    urllib.request.urlretrieve(url, output_path)
    print(f"Datei wurde heruntergeladen und unter '{output_path}' gespeichert.")
else:
    print(f"Datei bereits vorhanden unter '{output_path}' – überspringe Download.")

# Lade das Modell
print("Lade das Word2Vec-Modell...")
model = KeyedVectors.load_word2vec_format(output_path, binary=True)
print("Modell erfolgreich geladen.")


## Anwendung der Worteinbettung auf eine Textklassifizierungsaufgabe
Wir haben nun unsere Wortrepräsentation, einen Vektor für jedes Wort in unserem Vokabular. Aber wir müssen uns mit ganzen Sätzen befassen, also müssen wir eine Satzeinbettung erstellen, d.h. wir brauchen einen Vektor, der den ganzen Satz repräsentiert, und jedes Merkmal im Vektor wird auf den Worteinbettungen basieren. Da es viele Möglichkeiten gibt und wir dieses Thema nicht behandeln werden, wenden wir eine sehr einfache Methode an: Der i-te Wert in der Satzeinbettung ist der Mittelwert der i-ten Werte in der Worteinbettung aller Wörter des Satzes.

Wir erstellen eine Klasse, die unser Vokabular und die Vektoren der Handschuhe enthält, und wandeln dann jede Rezension (in unserem Beispiel einen Satz) in eine Vektordarstellung um, wie wir es zuvor beschrieben haben.

In [ ]:
import re


# Minimaler Text-Cleaner
def clean_text(text):
    text = text.lower()
    text = re.sub(r"<.*?>", "", text)
    text = re.sub(r"[^a-zA-Z']", " ", text)
    return text


# Vektorisierer
class Word2VecVectorizer:
    def __init__(self, model):
        print("Loading in word vectors...")
        self.word_vectors = model
        print("Finished loading in word vectors.")

    def transform(self, data):
        D = self.word_vectors.vector_size
        X = np.zeros((len(data), D))
        emptycount = 0

        for i, sentence in enumerate(data):
            tokens = clean_text(sentence).split()
            vecs = []
            for word in tokens:
                try:
                    vecs.append(self.word_vectors[word])
                except KeyError:
                    continue
            if len(vecs) > 0:
                X[i] = np.mean(vecs, axis=0)
            else:
                emptycount += 1
        print(f"Samples with no known words: {emptycount}/{len(data)}")
        return X


Als Nächstes erstellen wir ein Vectorizer-Objekt, mit dessen Hilfe wir unsere Bewertungen in Vektoren, also eine numerische Darstellung, umwandeln können. Mit diesen Vektoren können wir dann unseren Klassifikator füttern.

In [ ]:
# Vektorisierung
vectorizer = Word2VecVectorizer(model)
X_train = vectorizer.transform(X_train_raw)
X_test = vectorizer.transform(X_test_raw)


## Trainieren eines Klassifikators auf den Satzeinbettungen

In [ ]:
from sklearn.ensemble import RandomForestClassifier

# Klassifikation
clf = RandomForestClassifier(n_estimators=200, random_state=42)
clf.fit(X_train, Y_train)


In [ ]:
# Visualisierungen
def plot_confusion_matrix(y_true, y_pred):
    cm = confusion_matrix(y_true, y_pred)
    df_cm = pd.DataFrame(cm, index=[0, 1], columns=[0, 1])
    sn.heatmap(df_cm, annot=True, fmt="d", cmap="Blues")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.title("Confusion Matrix")
    plt.show()


def plot_roc_curve(y_true, y_pred):
    fpr, tpr, _ = roc_curve(y_true, y_pred)
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, label=f"AUC = {roc_auc:.2f}")
    plt.plot([0, 1], [0, 1], "r--")
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("ROC Curve")
    plt.legend(loc="lower right")
    plt.show()


In [ ]:
# Ergebnisse
print("Train score:", clf.score(X_train, Y_train))
print("Test score:", clf.score(X_test, Y_test))


In [ ]:
# Vorhersagen
y_pred = clf.predict(X_test)
print(metrics.classification_report(Y_test, y_pred, digits=5))
plot_confusion_matrix(Y_test, y_pred)
plot_roc_curve(Y_test, y_pred)


# Where's the intelligence?

- Was ist smart an diesen Word-Embeddings?
- Lesen Sie bei Chris Manning, wie die GloVe genau funktioniern: https://nlp.stanford.edu/projects/glove/
